In [1]:
import pandas as pd
import anthropic
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
import json

In [2]:
with open('./data/documents.json', 'r') as f_in:
    documents = json.load(f_in)

In [3]:
documents[0]

{'id': '3ba4d080-97a6-4954-829e-121e008c43e9',
 'page_content': 'In the `hfla-site` Slack channel, send an introductory message with your GitHub handle/username asking to be added to the Hack for LA website GitHub repository (this repository).  \n**NOTE:** Once you have accepted the GitHub invite (comes via email or in your GitHub notifications), **please do the following**:  \n1. Make your own Hack for LA GitHub organization membership public by following this [guide](https://help.github.com/en/articles/publicizing-or-hiding-organization-membership#changing-the-visibility-of-your-organization-membership).\n2. Set up two-factor authentication on your account by following this [guide](https://docs.github.com/en/github/authenticating-to-github/configuring-two-factor-authentication).  \n***',
 'header_1': '**How to Contribute to Hack for LA**',
 'header_2': '**Part 1: Setting up the development environment**',
 'header_3': '**1.1 Dev setup (1): Join the repository team**'}

## Indexing Stage

In [4]:
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

/opt/homebrew/Caskroom/miniconda/base/envs/llm_venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
query = 'Where do I find github issues to work on?'

In [6]:
v = model.encode(query)

In [7]:
len(v)

384

In [8]:
index_settings = {
  "settings": {
    "number_of_shards": 1,
    "number_of_replicas": 0
  },
  "mappings": {
    "dynamic": True,
    "properties": {
      "id": { "type": "keyword" },
      "page_content": { "type": "text" },
      "header_1": { "type": "text" },
      "header_2": { "type": "text" },
      "header_3": { "type": "text" },
      "header_4": { "type": "text"},
      "header_5": { "type": "text"},
      "page_content_vector": {
        "type": "dense_vector",
        "dims": 384,
        "index": True,
        "similarity": "cosine"
      },
     "metadata_vector": {
        "type": "dense_vector",
        "dims": 384,
        "index": True,
        "similarity": "cosine"
      },
     "combined_vector": {
        "type": "dense_vector",
        "dims": 384,
        "index": True,
        "similarity": "cosine"
      }
    }
  }
}

index_name = "contributing_h4la"

In [9]:
es_client = Elasticsearch('http://localhost:9200')

In [10]:
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'contributing_h4la'})

In [11]:
for doc in tqdm(documents):
    # extract content from doc
    content = doc.get('page_content')
    headers = ' '.join([doc.get(f'header_{i}', '') for i in range(1, 6)])

    # combine headers and content for full text encoding
    combined_text = headers + ' ' + content

    # encode content and headers
    doc['page_content_vector'] = model.encode(content)
    doc['metadata_vector'] = model.encode(headers)
    doc['combined_vector'] = model.encode(combined_text)

  0%|          | 0/60 [00:00<?, ?it/s]

In [12]:
for doc in tqdm(documents):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

  0%|          | 0/60 [00:00<?, ?it/s]

## Retrieval Evaluation Stage

In [13]:
from typing import Dict
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_elasticsearch import ElasticsearchRetriever

In [14]:
es_url = 'http://localhost:9200'

In [15]:
embeddings = HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-cos-v1")

/opt/homebrew/Caskroom/miniconda/base/envs/llm_venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [16]:
# will return keyword search dictionary and vector search dictionary

def hybrid_query(search_query: str) -> Dict:
    vector = embeddings.embed_query(search_query)  # same embeddings as for indexing
    
    return {
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": search_query,
                        "fields": ["page_content", "header_1", "header_2", "header_3", "header_4", "header_5"],
                        "type": "best_fields",
                        "boost": 0.5,
                    }
                },
            }
        },
        "knn": {
            "field": "combined_vector",
            "query_vector": vector,
            "k": 5,
            "num_candidates": 10000,
            "boost": 0.5,
        },
        "size": 5, # output the size
    }

In [17]:
hybrid_retriever = ElasticsearchRetriever.from_es_params(
    index_name=index_name,
    body_func=hybrid_query,
    content_field='page_content',
    url=es_url,
)

In [18]:
# produces the results => list of the answers
hybrid_results = hybrid_retriever.invoke(query)

In [19]:
# print(hybrid_results[0].metadata['_source'])

for result in hybrid_results:
    print(result.metadata['_source']['header_1'],
          result.metadata['_source']['header_2'],
          result.metadata['_source']['header_3'],
          result.metadata['_source'].get('header_4', ''),
          result.metadata['_source'].get('header_5', ''),
          result.metadata['_score']
         )

**How to Contribute to Hack for LA** **Part 3: Pull Requests** **3.1 How to make a pull request** **3.1.b Complete pull request on Hack for LA `website` repo** **vi. After pull request is submitted/merged** 9.00024
**How to Contribute to Hack for LA** **Part 2: How the Website team works with GitHub issues** **2.3 Where can I find GitHub issues to work on?**   7.21489
**How to Contribute to Hack for LA** **Part 2: How the Website team works with GitHub issues** **2.3 Where can I find GitHub issues to work on?** **2.3.a Available issues for new members**  7.202876
**How to Contribute to Hack for LA** **Part 2: How the Website team works with GitHub issues** **2.3 Where can I find GitHub issues to work on?** **2.3.d What if you see bugs/errors that are not connected to an issue?**  7.1937537
**How to Contribute to Hack for LA** **Part 2: How the Website team works with GitHub issues** **2.3 Where can I find GitHub issues to work on?** **2.3.b Available issues for returning members**  7.1

In [20]:
print(hybrid_results[0].page_content)

**NOTE**: After completing your assignment and committing all of the changes, you must leave your current branch and return to the `gh-pages` branch.  
Run the following command to return to the `gh-pages` branch:  
```bash
git checkout `gh-pages`
```
Once your pull request is merged you can delete your branch with the following command:  
```bash
git branch -d update-give-link-2093
```  
Now you are all set to work on a new PR. Start over at [**2.3 Where can I find GitHub issues to work on?**](#23-where-can-i-find-github-issues-to-work-on) and repeat completing parts 2 and 3.


## Hybrid Search

In [21]:
df_ground_truth = pd.read_csv('./data/ground-truth-retrieval.csv')

In [22]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [23]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [24]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [25]:
def elastic_search_hybrid(field, query):
    def hybrid_query(search_query: str) -> Dict:
        vector = embeddings.embed_query(search_query)
        
        return {
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": search_query,
                            "fields": ["page_content", "header_1", "header_2", "header_3", "header_4", "header_5"],
                            "type": "best_fields",
                            "boost": 0.3,
                        }
                    },
                }
            },
            "knn": {
                "field": field,
                "query_vector": vector,
                "k": 5,
                "num_candidates": 1000,
                "boost": 5,
            },
            "size": 5,
            "_source": ["page_content", "header_1", "header_2", "header_3", "header_4", "header_5", "id"],
        }
    
    
    hybrid_retriever = ElasticsearchRetriever.from_es_params(
        index_name=index_name,
        body_func=hybrid_query,
        content_field='page_content',
        url=es_url,
    )

    hybrid_results = hybrid_retriever.invoke(query)
    
    result_docs = []
    
    for hit in hybrid_results:
        result_docs.append(hit.metadata['_source'])

    return result_docs

In [26]:
question = ground_truth[0]['question']

In [27]:
print(question)

How do I request access to the Hack for LA website GitHub repository?


In [28]:
elastic_search_hybrid("combined_vector", question)

[{'id': '3ba4d080-97a6-4954-829e-121e008c43e9',
  'header_1': '**How to Contribute to Hack for LA**',
  'header_2': '**Part 1: Setting up the development environment**',
  'header_3': '**1.1 Dev setup (1): Join the repository team**'},
 {'id': '7da5e900-3183-4d40-b0af-021562c21707',
  'header_1': '**How to Contribute to Hack for LA**',
  'header_2': '**Part 1: Setting up the development environment**',
  'header_3': '**1.4 Dev setup (4): Clone (Create) a copy on your computer**',
  'header_4': '**1.4.c What if you accidentally cloned using the repository URL from the HackForLA Github (instead of the fork on your Github)?**',
  'header_5': '**i. Resolve remote (1): reset `origin` remote url**'},
 {'id': '729f9112-7dc4-4ad3-91c0-498578f7df13',
  'header_1': '**How to Contribute to Hack for LA**',
  'header_2': '**Part 1: Setting up the development environment**',
  'header_3': '**1.4 Dev setup (4): Clone (Create) a copy on your computer**',
  'header_4': '**1.4.c What if you accidentally

In [29]:
def question_text_hybrid(q):
    question = q['question']

    return elastic_search_hybrid('combined_vector', question)

In [30]:
def evaluate(ground_truth, search_function):
    relevance_total = []
    
    # relevance is matching the id from the questions to the id in the documents
    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [31]:
evaluate(ground_truth, question_text_hybrid)

  0%|          | 0/242 [00:00<?, ?it/s]

{'hit_rate': 0.8925619834710744, 'mrr': 0.7353305785123967}

## Hybrid Search with RRF / ElasticSearch

In [32]:
def compute_rrf(rank, k=60):
    """ Own implementation of the relevance score """
    
    return 1 / (k + rank)

In [98]:
def elastic_search_hybrid_rrf(field, query, vector, k=60):
    knn_query = {
        "knn": {
            "field": field,
            "query_vector": vector,
            "k": 10,
            "num_candidates": 1000,
            "boost": 0.5,
        },
        "size": 10
    }
    
    keyword_query = {
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["page_content^2", "header_1", "header_2", "header_3", "header_4", "header_5"],
                "type": "best_fields",
                "boost": 0.5,
            }
        },
        "size": 10
    }
    
    # Perform searches
    knn_results = es_client.search(index=index_name, body=knn_query)['hits']['hits']
    keyword_results = es_client.search(index=index_name, body=keyword_query)['hits']['hits']
    
    # Apply RRF
    rrf_scores = {}
    
    for rank, hit in enumerate(knn_results):
        doc_id = hit['_id']
        rrf_scores[doc_id] = compute_rrf(rank + 1, k)
    
    for rank, hit in enumerate(keyword_results):
        doc_id = hit['_id']
        
        if doc_id in rrf_scores:
            rrf_scores[doc_id] += compute_rrf(rank + 1, k)
        else:
            rrf_scores[doc_id] = compute_rrf(rank + 1, k)
    
    # Sort and get top results
    reranked_docs = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
    
    final_results = []
    
    for doc_id, score in reranked_docs[:5]:
        doc = es_client.get(index=index_name, id=doc_id)
        
        final_results.append(doc['_source'])


    return final_results

In [34]:
def question_text_hybrid_rrf(q):
    question = q['question']
    v_q = model.encode(question)

    return elastic_search_hybrid_rrf('page_content_vector', question, v_q)

In [35]:
evaluate(ground_truth, question_text_hybrid_rrf)

  0%|          | 0/242 [00:00<?, ?it/s]

{'hit_rate': 0.8966942148760331, 'mrr': 0.719077134986226}

## RAG Flow

In [36]:
client = anthropic.Anthropic()

In [37]:
def search_hyrid_rrf(q):
    v_q = model.encode(q)

    return elastic_search_hybrid_rrf('page_content_vector', q, v_q)

In [38]:
prompt_template = """
You're an assistant to an open source software engineering project on github. Answer the QUESTION based on
the CONTEXT from our contributor FAQ database.
Use only the facts and relevant hyperlinks, if any, from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
page_content: {page_content}
""".strip()

In [39]:
def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [40]:
def llm(prompt):
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1024,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    # print('token usage: ', response.usage) # get tokens info
    # print('==============================')
    return response.content[0].text

In [41]:
def rag(query):
    search_results = search_hyrid_rrf(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [42]:
answer = rag(query)

print(answer)

The best way to find GitHub issues to work on is through Hack for LA's GitHub Project Board. Here are the key points:

1. Visit the [GitHub Project Board](https://github.com/orgs/hackforla/projects/86).

2. Focus on the `Prioritized Backlog` column, which contains approved and prioritized issues.

3. The issues in this column are filtered so that the topmost issue has the highest priority and should be worked on next.

4. For newcomers or those looking for simpler tasks, you can use the [`good first issues`](https://github.com/orgs/hackforla/projects/86/views/2) link to display beginner-friendly issues for both front-end and back-end roles in the Prioritized Backlog column.

Remember to check the Project Board regularly, as it's the primary source for available and prioritized tasks.


## RAG Evaluation - LLM as a Judge (Claude 3.5 Sonnet)

In [43]:
prompt2_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [44]:
len(ground_truth)

242

In [45]:
record = ground_truth[10]['question']

print('Question: ', record)

Question:  How do I create a copy of the Hack for LA website repository on my GitHub account?


In [46]:
answer_llm = rag(record)

print(answer_llm)

To create a copy of the Hack for LA website repository on your GitHub account, you need to fork the repository. Here's how you can do it:

1. Go to the hackforla/website repository on GitHub.

2. Click on the "Fork" button located at the top right corner of the page. You can find it here: <a href="https://github.com/hackforla/website/fork"><button><img src="https://user-images.githubusercontent.com/17777237/54873012-40fa5b00-4dd6-11e9-98e0-cc436426c720.png" width="8px"> Fork</button></a>

3. If prompted, select where you want to fork the repository (usually your personal GitHub account).

4. GitHub will create a copy of the repository under your account. The new URL will look like this: `https://github.com/<your_GitHub_user_name>/website`

It's important to note that this creates a remote copy of the repository on your GitHub account. It is not yet on your local machine.

After forking, you should also:

1. Make your Hack for LA GitHub organization membership public by following this [

In [47]:
prompt = prompt2_template.format(question=record, answer_llm=answer_llm)

In [48]:
print(prompt)

You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: How do I create a copy of the Hack for LA website repository on my GitHub account?
Generated Answer: To create a copy of the Hack for LA website repository on your GitHub account, you need to fork the repository. Here's how you can do it:

1. Go to the hackforla/website repository on GitHub.

2. Click on the "Fork" button located at the top right corner of the page. You can find it here: <a href="https://github.com/hackforla/website/fork"><button><img src="https://user-images.githubusercontent.com/17777237/54873012-40fa5b00-4dd6-11e9-98e0-cc436426c720.png" width="8px"> Fork</button></a>

3. If prompted, select where you want to fork the repository (usually your personal GitHub account).

4. Gi

In [49]:
answer_test = llm(prompt)

In [62]:
json.loads(answer_test)

# print(answer_test)

{'Relevance': 'RELEVANT',
 'Explanation': "The generated answer directly addresses the question by providing a step-by-step guide on how to create a copy (fork) of the Hack for LA website repository on the user's GitHub account. It includes specific instructions, the correct repository name, and even provides additional relevant information such as making organization membership public and setting up two-factor authentication. The answer is comprehensive and precisely tailored to the question asked."}

In [53]:
df_sample = df_ground_truth.sample(n=100, random_state=1)

In [54]:
df_sample.head()

,id,question
67,7900ab8b-86e3-4a25-94d6-568f12529a0f,How are the issues in the Prioritized Backlog ...
241,ae00f395-3da6-4329-b048-d34236f2f76e,Are there any official guides or documentation...
205,a0235484-04f5-4db2-8fe1-5679fd0d9659,Are there any guidelines for summarizing the p...
122,5e6a3d13-236b-4a65-928a-a123ee8eefe5,How do I update my gh-pages branch with the la...
89,8fa8f710-1645-40d3-a6c4-22f5c6db766b,What steps should I follow to claim a task in ...


In [55]:
sample = df_sample.to_dict(orient='records')

In [56]:
print(sample[:3])

[{'id': '7900ab8b-86e3-4a25-94d6-568f12529a0f', 'question': 'How are the issues in the Prioritized Backlog column organized, and why is this important for contributors?'}, {'id': 'ae00f395-3da6-4329-b048-d34236f2f76e', 'question': 'Are there any official guides or documentation for setting up Docker and Docker Compose in our development environment?'}, {'id': 'a0235484-04f5-4db2-8fe1-5679fd0d9659', 'question': 'Are there any guidelines for summarizing the purpose and impact of changes made in a pull request for updating project information on the Hack for LA website?'}]


In [63]:
evaluations = []

for record in tqdm(sample):
    question = record['question']
    answer_llm = rag(question) 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)

    evaluations.append((record, answer_llm, evaluation))

  0%|          | 0/100 [00:00<?, ?it/s]

In [67]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

In [79]:
def safe_parse(item):
    if isinstance(item, dict):
        return item
    try:
        return json.loads(item)
    except:
        return {}

In [80]:
df_eval['id'] = df_eval.record.apply(lambda d: safe_parse(d).get('id'))
df_eval['question'] = df_eval.record.apply(lambda d: safe_parse(d).get('question'))
df_eval['relevance'] = df_eval.evaluation.apply(lambda d: safe_parse(d).get('Relevance'))
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: safe_parse(d).get('Explanation'))

del df_eval['record']
del df_eval['evaluation']

In [81]:
df_eval.relevance.value_counts(normalize=True)

relevance
RELEVANT           0.89
PARTLY_RELEVANT    0.10
NON_RELEVANT       0.01
Name: proportion, dtype: float64

In [82]:
df_eval.to_csv('./data/rag-eval-claude-3-5-sonnet.csv', index=False)

In [83]:
df_eval.head()

,answer,id,question,relevance,explanation
0,The issues in the Prioritized Backlog column a...,7900ab8b-86e3-4a25-94d6-568f12529a0f,How are the issues in the Prioritized Backlog ...,RELEVANT,The generated answer directly addresses the qu...
1,"Based on the provided context, here are the of...",ae00f395-3da6-4329-b048-d34236f2f76e,Are there any official guides or documentation...,RELEVANT,The generated answer directly addresses the qu...
2,"Yes, there are guidelines for summarizing the ...",a0235484-04f5-4db2-8fe1-5679fd0d9659,Are there any guidelines for summarizing the p...,RELEVANT,The generated answer directly addresses the qu...
3,To update your gh-pages branch with the latest...,5e6a3d13-236b-4a65-928a-a123ee8eefe5,How do I update my gh-pages branch with the la...,RELEVANT,The generated answer directly addresses the qu...
4,To claim a task in the project's GitHub reposi...,8fa8f710-1645-40d3-a6c4-22f5c6db766b,What steps should I follow to claim a task in ...,RELEVANT,The generated answer directly addresses the qu...


In [84]:
df_eval[df_eval.relevance == 'NON_RELEVANT']

,answer,id,question,relevance,explanation
41,The correct command to clone the Hack for LA w...,ac8e0f36-ea98-4ce0-badc-4b1195f83651,What's the correct command to clone the Hack f...,NON_RELEVANT,The generated answer provides instructions for...


### RAG Evaluation 2: Claude 3 Haiku as LLM as a Judge

In [85]:
def llm_haiku(prompt):
    response = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1024,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    # print('token usage: ', response.usage) # get tokens info
    # print('==============================')
    return response.content[0].text

In [86]:
def rag_haiku(query):
    search_results = search_hyrid_rrf(query)
    prompt = build_prompt(query, search_results)
    answer = llm_haiku(prompt)

    return answer

In [87]:
evaluations_haiku = []

for record in tqdm(sample):
    question = record['question']
    answer_llm = rag_haiku(question) 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = safe_parse(llm_haiku(prompt))
    
    evaluations_haiku.append((record, answer_llm, evaluation))

  0%|          | 0/100 [00:00<?, ?it/s]

In [90]:
df_eval_2 = pd.DataFrame(evaluations_haiku, columns=['record', 'answer', 'evaluation'])

In [92]:
df_eval_2['id'] = df_eval_2.record.apply(lambda d: d['id'])
df_eval_2['question'] = df_eval_2.record.apply(lambda d: d['question'])

df_eval_2['relevance'] = df_eval_2.evaluation.apply(lambda d: d['Relevance'])
df_eval_2['explanation'] = df_eval_2.evaluation.apply(lambda d: d['Explanation'])

del df_eval_2['record']
del df_eval_2['evaluation']

In [93]:
df_eval_2.relevance.value_counts()

relevance
RELEVANT           99
PARTLY_RELEVANT     1
Name: count, dtype: int64

In [94]:
df_eval_2.relevance.value_counts(normalize=True)

relevance
RELEVANT           0.99
PARTLY_RELEVANT    0.01
Name: proportion, dtype: float64

In [96]:
df_eval_2.to_csv('./data/rag-eval-claude-3-haiku.csv', index=False)

In [97]:
df_eval_2.head()

,answer,id,question,relevance,explanation
0,Based on the information provided in the CONTE...,7900ab8b-86e3-4a25-94d6-568f12529a0f,How are the issues in the Prioritized Backlog ...,RELEVANT,The generated answer provides a comprehensive ...
1,Based on the information provided in the CONTE...,ae00f395-3da6-4329-b048-d34236f2f76e,Are there any official guides or documentation...,RELEVANT,The generated answer directly addresses the qu...
2,Based on the context provided from the Hack fo...,a0235484-04f5-4db2-8fe1-5679fd0d9659,Are there any guidelines for summarizing the p...,RELEVANT,The generated answer provides detailed guideli...
3,Based on the information provided in the conte...,5e6a3d13-236b-4a65-928a-a123ee8eefe5,How do I update my gh-pages branch with the la...,RELEVANT,The generated answer provides a clear and comp...
4,Based on the information provided in the contr...,8fa8f710-1645-40d3-a6c4-22f5c6db766b,What steps should I follow to claim a task in ...,RELEVANT,The generated answer provides a comprehensive ...
